In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
dataset = pd.read_csv('IMDB Dataset.csv')

In [3]:
 dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
import re
import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer

In [5]:
dataset.shape

(50000, 2)

In [6]:
dataset['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [7]:
dataset['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [0]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [0]:
X = []
for i in range(0, 50000):
    review = remove_tags(dataset['review'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = re.sub(r"\s+[a-zA-Z]\s+", ' ', review)
    review = re.sub(r'\s+', ' ', review)
    X.append(review)

In [0]:
y = dataset.iloc[:, 1].values

In [0]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer(num_words=5000)

In [0]:
tokenizer.fit_on_texts(X_train)

In [0]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [0]:
#maxlen = max([len(s.split()) for s in X])

In [19]:
vocab_size

92285

In [0]:
X_train = pad_sequences(X_train, padding='post', maxlen=100)
X_test = pad_sequences(X_test, padding='post', maxlen=100)

In [21]:
X_train.shape

(40000, 100)

In [22]:
X_test.shape

(10000, 100)

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

In [0]:
embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

In [0]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [0]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [0]:
from keras import Sequential
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Dense

In [32]:
model = Sequential()

In [33]:
model.add(Embedding(vocab_size, 100, input_length=100, trainable='False'))

In [0]:
from keras.layers import LSTM
model.add(LSTM(128))

In [0]:
#model.add(LSTM(128,input_shape=(128,3)))

In [0]:
#model.add(Flatten())

In [0]:
#model.add(Dense(250,activation='relu'))

In [0]:
model.add(Dense(1,activation='sigmoid'))

In [39]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [40]:
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=1, validation_split=0.2)




Train on 32000 samples, validate on 8000 samples
Epoch 1/2





32000/32000 [==============================] - 115s 4ms/step - loss: 0.4399 - acc: 0.7832 - val_loss: 0.3404 - val_acc: 0.8546
Epoch 2/2
32000/32000 [==============================] - 113s 4ms/step - loss: 0.3018 - acc: 0.8760 - val_loss: 0.3401 - val_acc: 0.8549


In [0]:
y_pred = model.predict(X_test)

In [0]:
y_pred = (y_pred > 0.5)

In [43]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          9228500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 9,345,877
Trainable params: 9,345,877
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [45]:
cm

array([[4043,  992],
       [ 499, 4466]])

In [46]:
(4043+4466)/10000

0.8509